# Bisulfite Sequencing Simulation 
## Simulations performed with BSBolt v1.3.4

In [1]:
import gzip
import io
import os
import pickle
import random
import subprocess

import joblib
from tqdm.notebook import tqdm

In [2]:
wd = os.getcwd() + '/'

In [3]:
hg38_fasta = f'~/hg38_lambda.fa.gz'

In [4]:
# import hg38 sequence

hg38 = {}

with io.BufferedReader(gzip.open(hg38_fasta, 'rb')) as fasta:
    chrom_seq = []
    chrom = None
    for b_line in tqdm(fasta):
        line = b_line.decode('utf-8')
        if '>' in line:
            if chrom:
                hg38[chrom] = chrom_seq
            chrom = line.strip().replace('>', '')
        else:
            chrom_seq.append(line.strip())
    hg38[chrom] = chrom_seq

In [ ]:
# take 2,000,000 BP from each chromosome

chunk_size = len(hg38['chr1'][0])
simulation_genome = {}
simulation_size = 2000000

for chrom in tqdm(list(hg38.keys())):
    if '_' in chrom or 'lambda' in chrom or 'chrM' in chrom or 'X' in chrom or 'Y' in chrom:
        continue
    chrom_seq = hg38[chrom]
    simulation_seq = []
    while len(simulation_seq) < int(simulation_size / chunk_size):
        rand_pos = random.randint(0, len(chrom_seq) - 1)
        random_seq = chrom_seq[rand_pos]
        if random_seq.upper().count('N') < 10:
            simulation_seq.append(random_seq.replace('N', '').replace('n', ''))
    simulation_genome[chrom] = ''.join(simulation_seq)
    

        
with open('sim_genome.fa', 'w') as sim:
    sim.write(f'>chr1Dup\n')
    sim.write(f'{simulation_genome["chr1"][1000:10000]}\n')   
    for chrom, seq in simulation_genome.items():
        sim.write(f'>{chrom}\n')
        sim.write(f'{seq}\n')    

## Simulate Different Alignment Scenarios 

### Import simulation genome

In [6]:
sim_genome = {}


with open('sim_genome.fa', 'r') as fasta:
    chrom_seq = []
    chrom = None
    for line in tqdm(fasta):
        if '>' in line:
            if chrom:
                sim_genome[chrom] = ''.join(chrom_seq)
                chrom_seq = []
            chrom = line.strip().replace('>', '')
        else:
            chrom_seq.append(line.strip())
    sim_genome[chrom] = ''.join(chrom_seq)


## Generate Simulation Profile for Bisulfite Sequencing Reads

In [4]:
sim_out = '~/BSBolt_Paper/simulated_reads/'

In [5]:
simulation_base = ['python3', '-m', 'BSBolt', 'Simulate']

In [6]:
def log_output(log):
    assert isinstance(log, list)
    def run_func(func):
        def inner(*args, **kwargs):
            output = func(*args, **kwargs)
            log.append(args[1])
            return output
        return inner
    return run_func

In [7]:
sim_log = []

In [8]:
@log_output(sim_log)
def simulate_reads(simulation_command, output_dir):
    subprocess.run(['mkdir', '-p', output_dir])
    sim_command = subprocess.run(simulation_command)
    if sim_command.returncode:
        print('Simulation Error')

In [9]:
sim_commands = []

#### Paired End Directional 

In [10]:
for read_length in [50,100,150]:
    command = []
    command.append(list(simulation_base) + ['-O', f'{sim_out}pe_directional_{read_length}/sim', '-PE', '-RL', f'{read_length}', 
                             '-RD', '20', '-MR', '0.005', '-SE', '0.005', '-overwrite',
                             '-G', f'{wd}sim_genome.fa'])
    command.append(f'{sim_out}pe_directional_{read_length}')
    sim_commands.append(command)

#### Single End Directional 

In [11]:
for read_length in [50,100,150]:
    command = []
    command.append(list(simulation_base) + ['-O', f'{sim_out}se_directional_{read_length}/sim', '-RL', f'{read_length}', 
                             '-RD', '20', '-MR', '0.005', '-SE', '0.005', '-overwrite',
                             '-G', f'{wd}sim_genome.fa'])
    command.append(f'{sim_out}se_directional_{read_length}')
    sim_commands.append(command)

#### Paired End Undirectional 

In [12]:
for read_length in [50,100,150]: 
    command = []
    pe_u = list(simulation_base) +  ['-O', f'{sim_out}pe_undirectional_{read_length}/sim', '-PE', '-RL', f'{read_length}', 
                                         '-RD', '30', '-MR', '0.005', '-SE', '0.005', '-overwrite',
                                        '-G', f'{wd}sim_genome.fa', '-U']
    command.append(pe_u)
    command.append(f'{sim_out}pe_undirectional_{read_length}')
    sim_commands.append(command)

## Single End Undirectional 

In [13]:
for read_length in [50,100,150]: 
    command = []
    pe_u = list(simulation_base) +  ['-O', f'{sim_out}se_undirectional_{read_length}/sim', '-RL', f'{read_length}', 
                                         '-RD', '30', '-MR', '0.005', '-SE', '0.005', '-overwrite',
                                        '-G', f'{wd}sim_genome.fa', '-U']
    command.append(pe_u)
    command.append(f'{sim_out}se_undirectional_{read_length}')
    sim_commands.append(command)

### Simulate Low Coverage PE Library

In [15]:
for read_length in [50,100,150]:
    command = []
    command.append(list(simulation_base) + ['-O', f'{sim_out}pe_low_coverage_directional_{read_length}/sim', '-RL', f'{read_length}', 
                             '-RD', '8', '-MR', '0.005', '-SE', '0.005', '-overwrite', '-PE',
                             '-G', f'{wd}sim_genome.fa'])
    command.append(f'{sim_out}pe_low_coverage_directional_{read_length}')
    sim_commands.append(command)

### Simulate Low Coverage SE Library

In [16]:
for read_length in [50,100,150]:
    command = []
    command.append(list(simulation_base) + ['-O', f'{sim_out}se_low_coverage_directional_{read_length}/sim', '-RL', f'{read_length}', 
                             '-RD', '8', '-MR', '0.005', '-SE', '0.005', '-overwrite',
                             '-G', f'{wd}sim_genome.fa'])
    command.append(f'{sim_out}se_low_coverage_directional_{read_length}')
    sim_commands.append(command)

### Simulate Low Coverage High Mutation Rate / Sequencing Error Library

In [17]:
for read_length in [50,100,150]:
    command = []
    command.append(list(simulation_base) + ['-O', f'{sim_out}pe_error_directional_{read_length}/sim', '-RL', f'{read_length}', 
                             '-RD', '8', '-MR', '0.01', '-SE', '0.02', '-overwrite', '-PE',
                             '-G', f'{wd}sim_genome.fa'])
    command.append(f'{sim_out}pe_error_directional_{read_length}')
    sim_commands.append(command)

## Simulate Reads

In [18]:
joblib.Parallel(n_jobs=16, verbose=10)(joblib.delayed(simulate_reads)(*cmd) for cmd in sim_commands)

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   3 out of  22 | elapsed: 20.4min remaining: 129.1min
[Parallel(n_jobs=16)]: Done   6 out of  22 | elapsed: 41.2min remaining: 109.9min
[Parallel(n_jobs=16)]: Done   9 out of  22 | elapsed: 48.3min remaining: 69.8min
[Parallel(n_jobs=16)]: Done  12 out of  22 | elapsed: 60.0min remaining: 50.0min
[Parallel(n_jobs=16)]: Done  15 out of  22 | elapsed: 69.6min remaining: 32.5min
[Parallel(n_jobs=16)]: Done  18 out of  22 | elapsed: 101.3min remaining: 22.5min
[Parallel(n_jobs=16)]: Done  22 out of  22 | elapsed: 138.9min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]